In [234]:
import pandas as pd
import numpy as np 

olympics = pd.read_csv('/Users/emilyvincett/Downloads/olympics.csv', index_col=0, skiprows=1)

for col in olympics.columns: 
    if col[:2]=='01': olympics.rename({col:'Gold'+col[4:]},axis=1,inplace=True)
    if col[:2]=='02': olympics.rename({col:'Silver'+col[4:]},axis=1,inplace=True)
    if col[:2]=='03': olympics.rename({col:'Bronze'+col[4:]},axis=1,inplace=True)
    if col[:1]=='№': olympics.rename({col:'#'+col[1:]},axis=1,inplace=True)

country_ID = olympics.index.str.split('\s\(')
country = country_ID.str[0]
country_ID = country_ID.str[1].str[0:3]
olympics['ID'] = country_ID
olympics.index = country
olympics.drop('Totals',axis=0,inplace=True)
olympics


,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yugoslavia,16,26,29,28,83,14,0,3,1,4,30,26,32,29,87,YUG
Independent Olympic Participants,1,0,1,2,3,0,0,0,0,0,1,0,1,2,3,IOP
Zambia,12,0,1,1,2,0,0,0,0,0,12,0,1,1,2,ZAM
Zimbabwe,12,3,4,1,8,1,0,0,0,0,13,3,4,1,8,ZIM


In [169]:
def who_won_the_most_gold():
    return olympics['Gold'].idxmax()

who_won_the_most_gold()

'United States'

In [174]:
def largest_difference_summer_winter():
    absolute_difference = (olympics['Gold'] - olympics['Gold.1']).abs()
    return absolute_difference.idxmax()

largest_difference_summer_winter()

'United States'

In [231]:
def largest_relative_winners():

    dual_winners = olympics[(olympics['Gold']> 0) & (olympics['Gold.1']> 0)]
    summer_gold = dual_winners['Gold']
    winter_gold = dual_winners['Gold.1']
    total = summer_gold + winter_gold
    relative = (summer_gold - winter_gold).abs()/total
    return relative.idxmax()

largest_relative_winners()

'Bulgaria'

In [239]:
def points():
    
    GP = olympics['Gold.2']*3
    SP = olympics['Silver.2']*2
    BP = olympics['Bronze.2']*1
    points = GP + SP + BP 
    olympics['Points'] = points
    return olympics['Points']

points()

Afghanistan                           2
Algeria                              27
Argentina                           130
Armenia                              16
Australasia                          22
                                   ... 
Yugoslavia                          171
Independent Olympic Participants      4
Zambia                                3
Zimbabwe                             18
Mixed team                           38
Name: Points, Length: 146, dtype: int64

In [287]:
def state_with_most_counties():
    
    import pandas as pd 
    import numpy as np 

    census = pd.read_csv('/Users/emilyvincett/Downloads/census.csv')
    census_copy = census.copy()
    # Take only SUMLEV (50)
    census_copy = census_copy[census_copy['SUMLEV']==50]
    census_copy.reset_index(drop=True)
    census_copy.set_index(['STNAME','CTYNAME'])
    return census_copy.groupby('STNAME')['CTYNAME'].count().idxmax()

state_with_most_counties()

'Texas'

In [383]:
def three_most_pop_states():
    
    import pandas as pd 
    import numpy as np 

    census = pd.read_csv('/Users/emilyvincett/Downloads/census.csv')
    census.head(20)
    census_copy = census.copy()
    census_copy = census_copy[census_copy['SUMLEV']==50]
    #census_copy.groupby(['STNAME','CTYNAME'])['CENSUS2010POP']
    census_copy = census_copy.sort_values(['STNAME','CENSUS2010POP'],ascending=False)
    census_copy = census_copy.nlargest(3, 'CENSUS2010POP')
    census_copy = census_copy.reset_index(drop=True)
    return list(census_copy[0:3]['STNAME'])

three_most_pop_states()

['California', 'Illinois', 'Texas']

In [485]:
def county_largest_pop_change():
    import pandas as pd
    import numpy as np 

    census= pd.read_csv('/Users/emilyvincett/Downloads/census.csv')
    census = census[census['SUMLEV']==50]
    col_to_keep = (['STNAME','CTYNAME','POPESTIMATE2010',
                'POPESTIMATE2011','POPESTIMATE2012',
                'POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015'])

    census = census[col_to_keep]
    census.reset_index()
    col_to_use =(['POPESTIMATE2010','POPESTIMATE2011',
              'POPESTIMATE2012','POPESTIMATE2013',
              'POPESTIMATE2014','POPESTIMATE2015'])

    census['max_pop'] = census[col_to_use].max(axis=1)
    census['min_pop'] = census[col_to_use].min(axis=1)
    census['change'] = (census['max_pop'] - census['min_pop']).abs()
    census['change'].argmax()
    return census.iloc[census['change'].argmax()]['CTYNAME']

county_largest_pop_change()

'Harris County'

In [567]:
def washington_counties():
    
    import pandas as pd
    import numpy as np 

    census = pd.read_csv('/Users/emilyvincett/Downloads/census.csv')
    census = census[census['SUMLEV']==50]

    col_to_keep = (['STNAME', 'CTYNAME','REGION','POPESTIMATE2014','POPESTIMATE2015'])
    census = census[col_to_keep]
    census = census[census['REGION']<=2]
    census = census[census['CTYNAME'].str.startswith('Washington')]
    census = census[census['POPESTIMATE2015']> census['POPESTIMATE2014']]

    col_to_keep = (['STNAME','CTYNAME'])
    census = census[col_to_keep]
    return census

washington_counties ()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
